### Step1 导入相关包

In [2]:
import json
from langchain_community.document_loaders import PyPDFDirectoryLoader

### Step2 加载数据

In [3]:
data = "../PowerMarketQA/"
# 这里为了做演示，随便找了一个pdf

In [4]:
# 加载外部知识
loader = PyPDFDirectoryLoader(data)
docs_before_split = loader.load()
# 过滤目录、附录和页码
docs= [doc.page_content.replace(str(doc.metadata['page']-7),'',1) for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 
docs[0]

'第一章 电力现货市场基础   \n             \n   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现\n电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。\n上面从组织和实操两个维度对电力市场进行了描绘。  \n通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场\n泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就\n已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广\n义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种\n地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网\n才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。  \n狭义的电力市场是指现代竞争性的电力市场， 《中国电力百科全书》定义的电力市场\n即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源\n的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确\n定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义\n的电力市场。  \n之所以称之为狭义的电力市场，主要原因有二。  \n1）专指现代电力市场，狭义的电力市场兴起于 20世纪 80年代，是在电力作为商品\n出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创'

### Step3 数据清洗、分割

In [5]:
#文本分割
# 提取所有文本
full_text=""
for doc in docs:
    full_text += doc
full_text

# 把文本按“问题-答案”分割
paragraphs = full_text.split('？ \n')

# 对分割文档进行清洗
for i in range(0,len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('电力现货市场 101 问','').replace('第一章 电力现货市场基础','').replace('第二章 国内电力现货市场建设关键问题','').replace('第三章 电力现货市场技术支持系统','').replace('\n','').replace(' ','')



### Step4 Question-Answer组合

In [6]:
# 获取问题列表
question_list = []
for i in range(0,len(paragraphs)-1):

    last_idx = paragraphs[i].rindex(str(i+1))
    ques_str = paragraphs[i][last_idx:]
    # print(ques_str)
    question_list.append(ques_str+'？')

# print(question_list)

# 获取答案列表
answer_list = []
for i in range(1,len(paragraphs)):
    if i == len(paragraphs)-1:
        answer_list.append(paragraphs[i])
    else:
        last_idx = paragraphs[i].rindex(str(i+1))
        ans_str = paragraphs[i][:last_idx]   
        answer_list.append(ans_str) 

# 将问题与答案进行组合
# QA_List = []
# for i in range(0,len(question_list)):
#     qa_json = {}
#     qa_json["question"] = question_list[i]
#     qa_json["answer"] = answer_list[i]
#     QA_List.append(qa_json)

# print(QA_List[0])

### Step5 输出为excel文件

In [8]:
from pandas import DataFrame

df = DataFrame({'question': question_list, 'answer': answer_list})
df.to_excel('电力现货市场101问.xlsx', sheet_name='sheet1', index=False)